In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from skimage.draw import disk
from skimage.measure import label, regionprops
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, pred, target):
        # Flatten tensors
        pred = pred.view(-1)
        target = target.view(-1)
        
        intersection = (pred * target).sum()
        dice_score = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        return 1 - dice_score
    
def false_positive_loss(predicted_masks, ground_truth_masks):
    # Focus on ground truth masks that are empty
    empty_mask = (ground_truth_masks.sum(dim=(1, 2, 3)) == 0).float()  # Batch size dimension
    false_positive = (predicted_masks * empty_mask.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1))
    return false_positive.mean()

def extra_region_penalty(predicted_masks, ground_truth_masks):
    non_overlap = predicted_masks * (1 - ground_truth_masks)  # Predicted regions outside GT
    return non_overlap.mean()

class CustomLoss(nn.Module):
    def __init__(self, base_loss, alpha=1.0, beta=1.0):
        super(CustomLoss, self).__init__()
        self.base_loss = base_loss
        self.alpha = alpha  # Weight for false positive penalty
        self.beta = beta    # Weight for extra region penalty

    def forward(self, predicted_masks, ground_truth_masks):
        base_loss = self.base_loss(predicted_masks, ground_truth_masks)
        fp_penalty = false_positive_loss(predicted_masks, ground_truth_masks)
        extra_penalty = extra_region_penalty(predicted_masks, ground_truth_masks)
        return base_loss + self.alpha * fp_penalty + self.beta * extra_penalty

class CircleDataset(Dataset):
    def __init__(self, num_samples=1000, image_size=(64, 64), max_circles=5):
        self.binary_masks, self.instance_masks = self.create_dataset(num_samples, image_size, max_circles)

    def __len__(self):
        return len(self.binary_masks)

    def __getitem__(self, idx):
        binary_mask = self.binary_masks[idx]
        instance_mask = self.instance_masks[idx]
        instance_mask = np.pad(instance_mask, ((0, 0), (0, 0), (0, 5 - instance_mask.shape[2])), constant_values=0)
        return torch.tensor(binary_mask, dtype=torch.float32).unsqueeze(0), torch.tensor(instance_mask, dtype=torch.float32)

    @staticmethod
    def create_dataset(num_samples=1000, image_size=(64, 64), max_circles=5):
        binary_masks = []
        instance_masks = []

        for _ in range(num_samples):
            num_circles = random.randint(1, max_circles)
            _, binary_mask, _, _ = generate_image_with_circles(image_size, num_circles)
            instances = generate_instance_masks(binary_mask)

            binary_masks.append(binary_mask)
            instance_stack = np.stack(instances, axis=-1) if instances else np.zeros((*image_size, 0))
            instance_masks.append(instance_stack)

        return np.array(binary_masks), instance_masks

def generate_image_with_circles(image_size=(64, 64), num_circles=5):
    """Generate an image with random circles and return the image and binary mask."""
    image = np.zeros(image_size, dtype=np.uint8)
    binary_mask = np.zeros(image_size, dtype=np.uint8)

    for _ in range(num_circles):
        radius = random.randint(4, 8)
        center = (
            random.randint(radius, image_size[0] - radius),
            random.randint(radius, image_size[1] - radius),
        )
        rr, cc = disk(center, radius, shape=image_size)
        binary_mask[rr, cc] = 1

    return image, binary_mask, [], []

def generate_instance_masks(binary_mask):
    """Generate instance masks from a binary mask."""
    labeled_mask = label(binary_mask)
    instance_masks = []
    for region in regionprops(labeled_mask):
        instance_mask = labeled_mask == region.label
        instance_masks.append(instance_mask)
    return instance_masks

def visualize_results(image, binary_mask, instance_masks):
    """Visualize the image, binary mask, and instance masks."""
    fig, axes = plt.subplots(1, 2 + len(instance_masks), figsize=(15, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    axes[1].imshow(binary_mask, cmap='gray')
    axes[1].set_title("Binary Mask")
    axes[1].axis("off")

    for i, instance_mask in enumerate(instance_masks):
        axes[2 + i].imshow(instance_mask, cmap='gray')
        axes[2 + i].set_title(f"Instance Mask {i + 1}")
        axes[2 + i].axis("off")

    plt.tight_layout()
    plt.show()

class TransformerSegmentationModel(nn.Module):   # 8,64   # 4,256  # 2,1024  # 1,4096
    def __init__(self, img_size=64, patch_size=4, num_patches=256, in_channels=1, out_channels=5, embed_dim=256, num_heads=8, num_layers=4):
        super(TransformerSegmentationModel, self).__init__()

        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
        self.flatten_dim = patch_size * patch_size * in_channels
        self.out_channels = out_channels

        self.embedding = nn.Linear(self.flatten_dim, embed_dim)
        self.position_embedding = nn.Parameter(torch.randn(1, num_patches, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(embed_dim, num_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)

        self.output_proj = nn.Linear(embed_dim, patch_size * patch_size * out_channels)
        
        #self.output_proj2 = nn.Linear(img_size,img_size)

    def forward(self, x):
        batch_size = x.size(0)
        
        #print('forwards pass')
        #print(x.shape)

        # Divide image into patches
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        x = x.permute(0, 2, 3, 1, 4, 5).contiguous()
        x = x.view(batch_size, self.num_patches, -1)
        
        #print(x.shape)

        # Embedding and positional encoding
        x = self.embedding(x) + self.position_embedding

        # Transformer
        x = self.transformer(x)
        
        #print(x.shape)

        # Output projection and reshape
        #x = self.output_proj(x)
        #x = x.view(batch_size, self.img_size // self.patch_size, self.img_size // self.patch_size, -1)
        
        #print(x.shape)

        
        #x = x.view(batch_size,self.img_size,self.out_channels,self.img_size)
        
        #x = x.permute(0, 3, 1, 2).contiguous()
        
        # Output projection
        x = self.output_proj(x)  # Shape: (batch_size, num_patches, patch_size * patch_size * out_channels)
        
        #print(x.shape)

        # Reshape into patches
        x = x.view(
            batch_size,
            self.img_size // self.patch_size,  # Number of patches along height
            self.img_size // self.patch_size,  # Number of patches along width
            self.patch_size,
            self.patch_size,
            self.out_channels,
        )  # Shape: (batch_size, h_patches, w_patches, patch_h, patch_w, out_channels)

        # Permute to combine patches into spatial dimensions
        x = x.permute(0, 1, 3, 2, 4, 5).contiguous()  # Shape: (batch_size, h_patches * patch_h, w_patches * patch_w, out_channels)
        
        #print(x.shape)
        
        x = x.view(batch_size, self.img_size, self.img_size, self.out_channels)  # Shape: (batch_size, img_size, img_size, out_channels)
        
        #print(x.shape)
        
        #print(x.shape)

        return torch.sigmoid(x)




# class TransformerSegmentationModel(nn.Module):
#     def __init__(self, img_size=64, patch_size=4, num_patches=256, in_channels=1, out_channels=3, embed_dim=640, num_heads=8, num_layers=12):
#         super(TransformerSegmentationModel, self).__init__()

#         self.img_size = img_size
#         self.patch_size = patch_size
#         self.num_patches = num_patches
#         self.embed_dim = embed_dim
#         self.out_channels = out_channels

#         # Replace nn.Linear with a convolutional embedding layer
#         self.embedding = nn.Conv2d(
#             in_channels=in_channels,
#             out_channels=embed_dim,
#             kernel_size=patch_size,
#             stride=patch_size
#         )

#         self.position_embedding = nn.Parameter(torch.randn(1, embed_dim, img_size // patch_size, img_size // patch_size))

#         encoder_layer = nn.TransformerEncoderLayer(embed_dim, num_heads)
#         self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)

#         self.output_proj = nn.Conv2d(
#             in_channels=embed_dim,
#             out_channels=out_channels,
#             kernel_size=1
#         )

#     def forward(self, x):
#         batch_size = x.size(0)

#         # Apply convolutional embedding to input
#         x = self.embedding(x)  # Shape: (batch_size, embed_dim, H', W')
#         x = x + self.position_embedding  # Add positional encoding

#         # Reshape for the Transformer: (batch_size, embed_dim, num_patches) -> (num_patches, batch_size, embed_dim)
#         x = x.flatten(2).permute(2, 0, 1)

#         # Transformer
#         x = self.transformer(x)  # Shape: (num_patches, batch_size, embed_dim)

#         # Reshape back: (num_patches, batch_size, embed_dim) -> (batch_size, embed_dim, H', W')
#         x = x.permute(1, 2, 0).view(batch_size, self.embed_dim, self.img_size // self.patch_size, self.img_size // self.patch_size)

#         # Output projection
#         x = self.output_proj(x)  # Shape: (batch_size, out_channels, H, W)

#         return torch.sigmoid(x)

    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
# Dataset and DataLoader
num_samples = 100
train_dataset = CircleDataset(num_samples=num_samples)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Model, Loss, Optimizer
model = TransformerSegmentationModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
num_epochs = 2000
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for binary_mask, instance_masks in train_loader:
        optimizer.zero_grad()
        
        binary_mask = binary_mask.to(device)  # Move binary_mask to device
        instance_masks = instance_masks.to(device) 
        
        outputs = model(binary_mask)
        
        #print('outputs')
        #print(outputs.shape)
        #print(instance_masks.shape)
        
        #bce_loss  = criterion(outputs, instance_masks)
        #dice_loss = DiceLoss()(outputs, instance_masks)
        #loss = bce_loss + dice_loss
        
        base_loss = nn.BCELoss()
        custom_loss = CustomLoss(base_loss, alpha=2.0, beta=10.0)

        # Compute the loss
        loss = custom_loss(outputs, instance_masks)
        
        
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    
    if epoch % 100 == 0:
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")
        
        with torch.no_grad():

            
            # Visualizing the binary mask
            binary_mask = binary_mask[0].squeeze(0).cpu().numpy()
            instance_masks = instance_masks[0].cpu().numpy()
            predicted_masks = outputs[0].cpu().numpy()

            fig, axes = plt.subplots(2, max(instance_masks.shape[-1], predicted_masks.shape[-1]) + 1, figsize=(18, 6))

            # Binary Mask
            axes[0, 0].imshow(binary_mask, cmap='gray')
            axes[0, 0].set_title("Binary Mask")
            axes[0, 0].axis("off")

            # Ground Truth Masks
            for i in range(instance_masks.shape[-1]):
                axes[0, i + 1].imshow(instance_masks[..., i], cmap='gray')
                axes[0, i + 1].set_title(f"GT Mask {i+1}")
                axes[0, i + 1].axis("off")

            # Predicted Masks
            for i in range(predicted_masks.shape[-1]):
                axes[1, i + 1].imshow(predicted_masks[..., i], cmap='gray')
                axes[1, i + 1].set_title(f"Pred Mask {i+1}")
                axes[1, i + 1].axis("off")

            # Sum of Predicted Masks
            predicted_sum = np.sum(predicted_masks, axis=-1)
            axes[1, 0].imshow(predicted_sum, cmap='gray')
            axes[1, 0].set_title("Sum of Predicted Masks")
            axes[1, 0].axis("off")

            # Layout adjustment
            plt.tight_layout()
            plt.show()